## Efficient Simulation in the Inventory Model


#### John Stachurski

As we saw, simulation can be time consuming, especially if we need to simulate many paths.  And it's often true that large sample sizes are necessary to get an accurate picture.  Let's look at speeding up our simulations using the [Numba](https://numba.pydata.org/) library from Anaconda.

In [1]:
import numpy as np
import numba

Parameters:

In [3]:
s, S = 10.0, 100.0
p = 0.4

### Simulating One Path

Let's look at simulating one path without any optimization.

In [4]:
def sim_one_inventory_path_simple(initial_x=50.0, 
                                  sim_length=10000):
        
    dvals = np.random.geometric(p, size=sim_length-1) - 1
    X = np.empty(sim_length)
    X[0] = initial_x
    
    for t, d in enumerate(dvals):
        x = X[t]
        if x <= s:
            X[t+1] = max(S - d, 0)
        else:
            X[t+1] = max(x - d, 0)
            
    return X

How fast does this run?

In [5]:
timeit X = sim_one_inventory_path_simple(sim_length=1000000)

672 ms ± 5.46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Now let's build a jitted version.

In [6]:
@numba.jit(nopython=True)
def sim_one_inventory_path_jitted(initial_x=50.0, sim_length=10000):
    
    dvals = np.random.geometric(p, size=sim_length-1) - 1
    X = np.empty(sim_length, dtype=np.int32)
    X[0] = initial_x
    
    for t, d in enumerate(dvals):
        x = X[t]
        if x <= s:
            X[t+1] = max(S - d, 0)
        else:
            X[t+1] = max(x - d, 0)
            
    return X

This is **much** faster:

In [7]:
timeit X = sim_one_inventory_path_jitted(sim_length=1000000)

43.2 ms ± 127 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Simulating Many Paths

Here's a jitted (and hence fast) piece of code for simulating many paths.

In [9]:
@numba.jit(nopython=True)
def sim_paths_jitted(initial_x=50.0, 
                     num_paths=50000,
                     sim_length=400):
    
    X = np.empty((num_paths, sim_length), dtype=np.int32)
    X[:, 0] = initial_x
    
    for t in range(sim_length-1):
        dvals = np.random.geometric(p, size=num_paths) - 1
        for m in range(num_paths):
            x, d = X[m, t], dvals[m]
            if x <= s:
                y = max(S - d, 0)
            else:
                y = max(x - d, 0)
            X[m, t+1] = y
    return X


Let's check it's doing what we think it should.

In [10]:
X = sim_paths_jitted(num_paths=6, sim_length=4)

In [11]:
X

array([[50, 45, 43, 40],
       [50, 49, 48, 48],
       [50, 48, 47, 46],
       [50, 47, 47, 46],
       [50, 46, 46, 41],
       [50, 49, 49, 46]], dtype=int32)

Now let's get a reading on the time.

In [13]:
timeit X = sim_paths_jitted()

1.29 s ± 25.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


We can do something similar using Numba's `@vectorize` decorator.

In [14]:
@numba.vectorize
def G_vectorized(x, d):
    if x <= s:
        y = max(S - d, 0)
    else:
        y = max(x - d, 0)
    return y


In [18]:
def sim_paths_vectorized(initial_x=50.0, 
                              num_paths=50000,
                              sim_length=400):
    
    X = np.empty((num_paths, sim_length), dtype=np.int32)
    X[:, 0] = initial_x

    for t in range(sim_length-1):
        dvals = np.random.geometric(p, size=num_paths) - 1
        X[:, t+1] = G_vectorized(X[:, t], dvals)
    return X

In [19]:
X = sim_paths_vectorized(num_paths=6, sim_length=4)

In [20]:
X

array([[50, 45, 44, 39],
       [50, 50, 49, 49],
       [50, 50, 49, 48],
       [50, 47, 46, 45],
       [50, 50, 47, 45],
       [50, 49, 48, 46]], dtype=int32)

Here's the timing, which is similar to the jitted version.

In [21]:
timeit X = sim_paths_vectorized()

1.1 s ± 12.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


We can also target the vectorized function for parallel execution.

In [24]:
@numba.vectorize('float64(float64, float64)', target='parallel')
def G_vectorized_parallel(x, d):
    if x <= s:
        y = max(S - d, 0)
    else:
        y = max(x - d, 0)
    return y

In [25]:
def sim_paths_vectorized_parallel(initial_x=50.0, 
                                      num_paths=50000,
                                      sim_length=400):
    
    X = np.empty((num_paths, sim_length), dtype=np.int32)
    X[:, 0] = initial_x
    
    for t in range(sim_length-1):
        dvals = np.random.geometric(p, size=num_paths) - 1
        X[:, t+1] = G_vectorized_parallel(X[:, t], dvals)
    return X

In [26]:
X = sim_paths_vectorized_parallel(num_paths=6, sim_length=4)

In [27]:
X

array([[50, 47, 44, 43],
       [50, 50, 44, 44],
       [50, 49, 49, 49],
       [50, 50, 48, 48],
       [50, 50, 49, 49],
       [50, 50, 49, 46]], dtype=int32)

The timing is not much improved because the individual tasks are relatively small, and hence multiple threads are not efficiently exploited.

In [28]:
timeit X = sim_paths_vectorized_parallel()

1.12 s ± 5.72 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Let's try for more efficient parallelization, where the work is divided into a smaller number of chunks.

In [29]:
@numba.jit(nopython=True, parallel=True)
def sim_paths_jitted_parallel(num_paths=50000, 
                              sim_length=400,
                              sections=500):
    
    X = np.empty((num_paths, sim_length), dtype=np.int32)

    section_size = num_paths // sections
    
    for i in numba.prange(sections):
        j = i * section_size
        X[j:j+section_size, :] = sim_paths_jitted(num_paths=section_size, 
                                                sim_length=sim_length)
    return X

In [30]:
X = sim_paths_jitted_parallel(num_paths=8, 
                              sim_length=10, 
                              sections=2)

In [31]:
X

array([[50, 50, 48, 45, 45, 44, 42, 41, 38, 37],
       [50, 48, 48, 48, 45, 43, 42, 42, 41, 41],
       [50, 50, 46, 46, 46, 43, 42, 42, 41, 30],
       [50, 47, 46, 44, 44, 44, 44, 42, 42, 42],
       [50, 50, 50, 49, 48, 45, 40, 38, 36, 36],
       [50, 50, 47, 47, 47, 47, 46, 46, 43, 40],
       [50, 49, 49, 49, 47, 46, 42, 40, 40, 39],
       [50, 50, 50, 46, 46, 46, 45, 43, 42, 42]], dtype=int32)

Now the speed up from parallelization is significant.

In [34]:
timeit X = sim_paths_jitted_parallel()

238 ms ± 4.92 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
